In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%matplotlib inline

import os
from functools import lru_cache

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

<IPython.core.display.Javascript object>

In [2]:
from showoff import dataset_path, images, augmentation_pipeline, get_results, fin_filter

print(f"Dataset is located at {dataset_path=}")
print(f"There are {len(images)} images")

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------
Segmentation Models: using `tf.keras` framework.
Dataset is located at dataset_path='/home/maciej/.keras/datasets'
There are 45 images


<IPython.core.display.Javascript object>

In [10]:
cache = dict()

def show_results(
    image=images,
    use_threshold=False,
    mask_original=False,
    mask_mode=[
        "prediction",
        "ground truth",
        "intersection",
        "prediction - gt",
        "gt - prediction",
    ],
    alpha=0.5,
    threshold=(0, 1, 0.01),
    zadanie=[3, 5]
):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20))
    if zadanie == 3:
        x, y, m, p = fin_filter(image)
    else:
        x, y, m, p = get_results(image)
    if use_threshold or mask_original and zadanie == 5:
        p = p >= threshold

    if mask_original:
        ax1.imshow(x)
        mask = np.copy(x)
        color = np.array([0, 0, 255])
        if mask_mode == "prediction":
            mask[p] = color
        if mask_mode == "ground truth":
            mask[y] = color
        if mask_mode == "intersection":
            mask[p & y] = color
        if mask_mode == "prediction - gt":
            mask[p & ~y] = color
        if mask_mode == "gt - prediction":
            mask[~p & y] = color
        ax2.imshow(x)
        ax2.imshow(mask, alpha=alpha)
        ax1.set_title("Original")
        ax2.set_title("Masked")
    else:
        ax1.imshow(y, cmap="gray")
        ax2.imshow(p, cmap="gray")
        ax1.set_title("Ground truth")
        ax2.set_title("Prediction")
    for ax in [ax1, ax2]:
        ax.axis("off")
        ax.set_xticks([])
    fig.tight_layout()
    
    if use_threshold or mask_original or zadanie == 3:
        key = (image, threshold, zadanie)
        if key in cache:
            report = cache[key]
        else:
            report = classification_report(y[m], p[m], zero_division=0)
            cache[key] = report
        print(report)


<IPython.core.display.Javascript object>

In [9]:
interact_manual(show_results, alpha=(0, 1, 0.01))

""

interactive(children=(Dropdown(description='image', options=('01_dr.JPG', '01_g.jpg', '01_h.jpg', '02_dr.JPG',…

''

<IPython.core.display.Javascript object>